In [ ]:
from flask import Flask, render_template, redirect, request, url_for, session, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from wtforms.widgets import TextArea
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime as dt

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///web.db'

db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'thisisasecretkey'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

    
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    
    def __repr__(self):
        return '<User %r>' %self.id

class NewPostForm(FlaskForm):
    title = StringField("Title:", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "Title"})
    content = StringField('Things you want to share:', widget=TextArea())
    submit = SubmitField("Add Post")
    
class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    content = db.Column(db.String(200), nullable=False)
    author = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    date_created = db.Column(db.DateTime, default=dt.utcnow)

    def __repr__(self):
        return '<Post %r>' % self.id

class Product(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.Text)
    price = db.Column(db.Text)
    category = db.Column(db.Text)
    description = db.Column(db.Text)
    img_url = db.Column(db.Text)
    
    def __repr__(self):
        return '<Product %r>' % self.id

db.create_all()


class JoinForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired()], render_kw={"placeholder": "Username"})
    email = StringField("Email", validators=[InputRequired()], render_kw={"placeholder": "example@gmail.com"})
    password = PasswordField("Password", validators=[InputRequired()], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address already exists. Please choose a different one.")

class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired()], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired()], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")
    

################################################################################################################

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/member")
def member():
    return render_template('member.html')

@app.route("/pay")
def pay():
    return render_template('pay.html')

################################################################################################################
#lists
@app.route("/guide")
def guide():
    return render_template('guide.html')

#sample
@app.route("/watch")
def watch():
    sqlalchemyObj = db.engine.execute('select * from movie ORDER BY Rating DESC')
    movie = []
    for i in sqlalchemyObj:
        movie.append(i)
    dataset = []
    dict={}
    for i in movie:
        dict['name'] = i[1]
        dict['runtime'] = i[2]
        dict['rating'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dataset.append(dict.copy())

    return render_template('to-watch.html', dataset=dataset)


@app.route("/read")
def read():
    sqlalchemyObj = db.engine.execute('select * from book ORDER BY Name')
    book = []
    for i in sqlalchemyObj:
        book.append(i)
    dataset = []
    dict={}
    for i in book:
        dict['name'] = i[1]
        dict['price'] = i[2]
        dict['rating'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dataset.append(dict.copy())

    return render_template('to-read.html', dataset=dataset)


@app.route("/listen")
def listen():
    sqlalchemyObj = db.engine.execute('select * from playlist ORDER BY Category DESC')
    playlist = []
    for i in sqlalchemyObj:
        playlist.append(i)
    dataset = []
    dict={}
    for i in playlist:
        dict['name'] = i[1]
        dict['by_whom'] = i[2]
        dict['category'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dataset.append(dict.copy())

    return render_template('to-listen.html', dataset=dataset)


################################################################################################################
#community
@app.route("/comm")
def comm():
    stories = Post.query.order_by(Post.date_created).all()
    return render_template('comm.html', stories=stories)

@app.route('/say', methods=['POST', 'GET'])
@login_required
def say():
    form = NewPostForm()
    if form.validate_on_submit():
        new_post = Post(title=form.title.data, content=form.content.data, author=current_user.username)
        db.session.add(new_post)
        db.session.commit()
        return redirect('/say')
    else:
        posts = Post.query.order_by(Post.date_created).all()
    return render_template('say.html', form=form, posts=posts)


@app.route('/delete/<int:id>')
@login_required
def delete(id):
    post_to_delete = Post.query.get_or_404(id)

    try:
        db.session.delete(post_to_delete)
        db.session.commit()
        return redirect("/say")
    except:
        return 'Fail to delete post.'

    
@app.route('/update/<int:id>', methods=['GET', 'POST'])
@login_required
def update(id):
    post = Post.query.get_or_404(id)

    if request.method == 'POST':
        post.content = request.form['content']

        try:
            db.session.commit()
            return redirect('/say')
        except:
            return 'Fail to update post.'

    else:
        return render_template('update.html', post=post)


@app.route("/story/<post_id>")
def story(post_id):
    story = Post.query.get_or_404(int(post_id))
    return render_template('story.html', story=story)

################################################################################################################

@app.route("/shop")
def shop():
    sqlalchemyObj = db.engine.execute('select * from product ORDER BY Price DESC')
    products = []
    for i in sqlalchemyObj:
        products.append(i)
    dataset = []
    dict={}
    for i in products:
        dict['id']= i[0]
        dict['name'] = i[1]
        dict['price'] = i[2]
        dict['img_url'] = i[5]
        dataset.append(dict.copy())
    return render_template('shop.html', dataset=dataset)
    
    
@app.route("/product/<product_id>")
def product(product_id):
    detail = Product.query.get_or_404(int(product_id))
    return render_template('product.html', detail=detail)

################################################################################################################

@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == "POST":
        user = User.query.filter_by(username=request.form['username']).first()
        if user:
            if bcrypt.check_password_hash(user.password, request.form['password']):
                login_user(user)
                return redirect("/")
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html')        


@app.route("/join", methods=["GET", "POST"])
def signup():
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        hashed_password = bcrypt.generate_password_hash(request.form['password'])
        new_user = User(username=username, email=email, password=hashed_password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect("/login")
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('join.html')

@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect("/login")

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

if __name__ == '__main__':
    app.run('localhost', 1124)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/sammy/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:1124/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 16:17:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 16:17:00] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 16:17:00] "GET /static/assets/img/support.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 16:17:00] "GET /static/assets/img/free.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 16:17:00] "GET /static/assets/img/love.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 16:17:03] "GET /static/assets/img/home.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 16:17:03] "GET /static/assets/heart.ico HTTP/1.1" 200 -
